# Welcome!
This notebook demonstrates how to develop a conversational system that uses a deep knowledge base about hotels, combining structured instance-level data and an ontological model. The knowledge graph (KG) and ontology enable reasoning to enhance dialogue response generation. The task involves integrating a GraphRAG-like approach to query the knowledge base and generate accurate, context-aware responses.

Specifically, the notebook has the following steps:

1. **Setup**: Loading the knowledge graph, dialogues, and required libraries (e.g., OWLAPY).
2. **Analyzing the knowledge graph**: Exploring its structure and entities using OWLAPY queries.
3. **Extending the ontology**: Adding TBox information for expressive reasoning.
4. **Creating dialogues**: Create dialogues based on the examples. Write 5 simple dialogues and 5 more detailed ones to showcase different types of interactions.
5. **Combining ontology and KG data**: Deploying an OWL reasoner to perform class-expression queries.
6. **Query generation with LLMs**: Using an LLM (e.g., Llama3.2) to generate or assist in creating queries against the KG.
7. **Generating responses**: Summarizing retrieved data into dialogue responses using a KG-augmented RAG approach.
8. **Evaluation**: Assessing the system's performance using metrics like intersection-over-union scores.

## Assignment
The goal of this assignment is to develop a logic-enhanced conversational system that retrieves and reasons over domain knowledge to assist in dialogue response generation. You will focus on both the technical aspects of KG+ontology reasoning and the integration with LLMs for robust responses.

### Assignment Steps
1. **Analyze the provided knowledge graph and dialogues**:
   - Explore the KG's entities, properties, and relevance to the dialogues.
   - Identify opportunities where ontology reasoning enhances dialogue responses.
2. **Extend the ontology**:
   - Add expressive TBox information to support meaningful inferences.
3. **Deploy the reasoning environment**:
   - Use OWLAPY to combine the KG (as ABox) with the ontology for reasoning-based queries.
4. **Generate class-expression queries**:
   - Use instruction-based, few-shot prompting with Llama3.2 to produce or assist in creating the queries.
5. **Summarize results into dialogue responses**:
   - Apply KG-augmented RAG to generate user-facing answers based on reasoning results.
6. **Evaluate the system**:
   - Use appropriate metrics, including intersection-over-union scores for set-based answers.

## Report
Write a **5-page report** in LNCS format that includes:

1. **Introduction**: Background on conversational systems with LLMs and the role of reasoning over domain knowledge.
2. **Methodology**: A detailed description of your approach, including diagrams and examples.
3. **Results**: Evaluation findings from the implemented steps.
4. **Discussion**: Strengths and weaknesses of your approach, lessons learned, and potential improvements.

Make sure to use the following template: [Springer Lecture Notes in Computer Science](https://www.overleaf.com/latex/templates/springer-lecture-notes-in-computer-science/kzwwpvhwnvfj)


## Grading
Your work will be evaluated based on:

1. **Code Implementation (30%)**: Quality and functionality of the logic-enhanced conversational system.
2. **Report (70%)**: Depth of analysis and clarity in presenting methods, results, and lessons learned.

## Kaggle Environment Notes
To ensure smooth execution:
- Load the required data into `/kaggle/input/`.
- Use `/kaggle/working/` for saving temporary files.
- Turn on GPUs and internet connectivity when necessary, and follow best practices for resource management.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input director

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ontology-data/data.ttl
/kaggle/input/rdf-ontology/data.rdf


# Install packages

In [2]:
!pip install jpype1==1.5.2
!pip install owlapy==1.5.1 
!pip install ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.5/493.5 kB 9.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 73.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 79.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.2/587.2 kB 37.4 MB/s eta 0:00:00
  Created wheel for owlready2: filename=owlready2-0.49-py3-none-any.whl size=23742426 sha256=271dc93ecdfb20af699e976dd756ea679efb9a52d37c289e5a67c716f99045f9
  Stored in directory: /root/.cache/pip/wheels/43/fe/dc/a0de3c289cfd5923ece6524469d328950e14fa0c90b1088ffa
Successfully built owlready2


# Import libraries


In [3]:
from owlapy import manchester_to_owl_expression, dl_to_owl_expression
from owlapy.iri import IRI
from owlapy.owl_ontology import Ontology
from owlapy.owl_reasoner import SyncReasoner, StructuralReasoner


import matplotlib.pyplot as plt
from collections import Counter

# 1. Analyze the provided knowledge graph (data.ttl).

## Loading ontology

In [5]:
from pathlib import Path
from owlapy.iri import IRI
from owlapy.owl_ontology import Ontology

path = Path("/kaggle/input/rdf-ontology/data.rdf")

# hard sanity checks
print("Exists:", path.exists())
print("Size (bytes):", path.stat().st_size)

onto = Ontology(IRI.create(path.as_uri()), load=True)

print("Ontology loaded successfully.")


Exists: True
Size (bytes): 1071766
Ontology loaded successfully.


In [6]:
print("Classes:", len(list(onto.classes_in_signature())))
print("Object properties:", len(list(onto.object_properties_in_signature())))
print("Data properties:", len(list(onto.data_properties_in_signature())))
print("Individuals:", len(list(onto.individuals_in_signature())))


Classes: 34
Object properties: 9
Data properties: 0
Individuals: 1746


## Helpers for analysis ontology

In [ ]:
def get_label(entity, ontology) -> str:
    """
    Get the rdfs:label for an entity from the ontology.
    Falls back to extracting local name from IRI if no label exists.
    """
    # Get the IRI first
    if hasattr(entity, "get_iri") and callable(entity.get_iri):
        iri_obj = entity.get_iri()
    elif hasattr(entity, "iri"):
        iri_obj = entity.iri
    else:
        return str(entity)
    
    # Try to get rdfs:label from the underlying owlready2 ontology
    try:
        # Access the underlying owlready2 world
        owlready_onto = ontology._onto
        
        # Get IRI as string
        if hasattr(iri_obj, "as_str") and callable(iri_obj.as_str):
            iri_str = iri_obj.as_str()
        else:
            iri_str = str(iri_obj)
        
        # Search for entity in owlready2 ontology and get its label
        with owlready_onto:
            entity_obj = owlready_onto.world.search_one(iri=iri_str)
            if entity_obj and hasattr(entity_obj, 'label') and entity_obj.label:
                # Return first label (they're usually lists)
                labels = entity_obj.label
                if labels and len(labels) > 0:
                    return str(labels[0])
    except:
        pass
    
    # Fallback: extract local name from IRI
    if hasattr(iri_obj, "as_str") and callable(iri_obj.as_str):
        iri_str = iri_obj.as_str()
    else:
        iri_str = str(iri_obj)

    return iri_str.rstrip("/").rsplit("/", 1)[-1]

## Plotting and analysis

In [ ]:
plt.rcParams["figure.figsize"] = (12, 6)

reasoner = StructuralReasoner(onto) # Not an actual reasoner, from docs: Tries to check instances fast (but maybe incomplete).

# Pull signatures once
classes = list(onto.classes_in_signature())
object_props = list(onto.object_properties_in_signature())
data_props = list(onto.data_properties_in_signature())
individuals = list(onto.individuals_in_signature())

'''Quick ontology summary'''

print("=" * 70)
print("\nOntology summary\n")
print("=" * 70)

print(f"Classes:           {len(classes)}")
print(f"Object properties: {len(object_props)}")
print(f"Data properties:   {len(data_props)}")
print(f"Individuals:       {len(individuals)}")

plt.figure()
plt.bar(
    ["Classes", "ObjProps", "DataProps", "Individuals"],
    [len(classes), len(object_props), len(data_props), len(individuals)]
)
plt.title("Ontology Component Counts")
plt.ylabel("Count")
plt.show()

'''Individual type distribution'''

print("\n" + "=" * 70)
print("\nIndividual type distribution (direct/asserted)\n")
print("=" * 70)

type_counter = Counter()

for ind in individuals:
    ind_types = reasoner.types(ind, direct=True)
    for cls in ind_types:
        type_counter[get_label(cls, onto)] += 1

type_df = (
    pd.DataFrame(type_counter.items(), columns=["Class", "Count"])
    .sort_values("Count", ascending=False)
    .reset_index(drop=True)
)

print(f"Unique classes with >=1 typed instance: {len(type_df)}")
if len(type_df) > 0:
    print("\nTop 15 classes by instance count:")
    print(type_df.head(15).to_string(index=False))

    # Plot top typed classes
    top_n = min(20, len(type_df))
    top = type_df.head(top_n).iloc[::-1]
    plt.figure(figsize=(12, 8))
    plt.barh(top["Class"], top["Count"])
    plt.title(f"Top {top_n} Classes by Instance Count")
    plt.xlabel("Instances")
    plt.ylabel("Class")
    plt.show()

    top10 = int(type_df.head(min(10, len(type_df)))["Count"].sum())
    rest = int(type_df.iloc[10:]["Count"].sum()) if len(type_df) > 10 else 0

    plt.figure()
    plt.bar(["Top 10 classes", "All other classes"], [top10, rest])
    plt.title("Instance Concentration (Top 10 vs Rest)")
    plt.ylabel("Instances")
    plt.show()
else:
    print("\nNo asserted rdf:type/class assertions found.")

'''Class hierarchy structure'''

print("\n" + "=" * 70)
print("Class hierarchy structure")
print("=" * 70)

sub_counts = []
for c in classes:
    subs = reasoner.sub_classes(c, direct=False)
    subs = set(subs)
    if c in subs:
        subs.remove(c)
    sub_counts.append((get_label(c, onto), len(subs)))

sub_df = (
    pd.DataFrame(sub_counts, columns=["Class", "NumSubclasses"])
    .sort_values("NumSubclasses", ascending=False)
    .reset_index(drop=True)
)

print("\nTop 15 classes by number of subclasses (descendants):")
print(sub_df.head(15).to_string(index=False))

top_n = min(20, len(sub_df))
top = sub_df.head(top_n).iloc[::-1]
plt.figure(figsize=(12, 8))
plt.barh(top["Class"], top["NumSubclasses"])
plt.title(f"Top {top_n} Classes by Number of Subclasses (Descendants)")
plt.xlabel("Number of subclasses")
plt.ylabel("Class")
plt.show()

'''show full panda tables '''

if len(type_df) > 0:
    display(type_df.head(50))

display(sub_df.head(50))

print("\nAnalysis complete!")

# 2. Create a small ontology that can support expressive inference about hotels and analyse the dialogues (examples.txt).

# Create your own dialogues

Once you have created your ontology, use it as the foundation for designing dialogues. Study the examples in examples.txt to understand their structure and content. Then, create 10 dialogues of your own, ensuring a range of difficulty levels: 5 simple ones and 5 more challenging ones. These dialogues should illustrate how your ontology can support reasoning and should include references to the types of information modeled in your ontology.

In [7]:
# Create 10 dialoges based on the description
dialogue1: dict = {"dialogue": "List all hostels that offer free wifi", 
                  "expected_query": "Hostel and has facility value Free_Wifi"}
dialogue2: dict = {"dialogue": "List all hotels near a water body", 
                  "expected_query": "Hotel and near WaterBody"}
dialogue3: dict = {"dialogue": "Find a restaurant that has traditional food", 
                  "expected_query": "Restaurant and (restaurantType value Traditional)"}
dialogue4: dict = {"dialogue": "I want to live in a luxury hotel that has parking along with a swimming pool and sauna", 
                  "expected_query": "LuxuryHotel and (has facility value Parking Space) and (has facility value Swimming Pool) and (has facility value Sauna)"}
dialogue5: dict = {"dialogue": "Show me 4 star camping sites which have Free Wifi", 
                  "expected_query": "CampingSite and (has rating value 4_stars) and (has facility value FreeWifi)"}
dialogue6: dict = {"dialogue": "Give me any hotel in Rome", 
                  "expected_query": "Hotel and 'location' some (Neighbourhood and 'in city' value 'Rome')"}
dialogue7: dict = {"dialogue": "Give me all Accommodation options that are next to Sado River", 
                  "expected_query": "AccommodationEntities and location some (Neighbourhood and 'next to' value 'Sado River')"}
dialogue8: dict = {"dialogue": "Give me any Accommodation next to water body or that has swimming pool", 
                  "expected_query": "AccommodationEntities and 'next to' some WaterBody or 'has facility' value 'Swimming Pool'"}
dialogue9: dict = {"dialogue": "Any accommodation in Lisbon that is in location that has Train Station", 
                  "expected_query": "AccommodationEntities and location some (Neighbourhood and inverse 'location' some Trainstation and 'in city' value 'Lisbon')"}
dialogue10: dict = {"dialogue": "Accommodation that is next to Nantes and has at least 4 facilities", 
                  "expected_query": "AccommodationEntities and location some (Neighbourhood and 'next to' value Nantes)  and 'has facility' min 4 owl:Thing"}
dialogues: list = [dialogue1, dialogue2, dialogue3, dialogue4, dialogue5, dialogue6, dialogue7, dialogue8, dialogue9, dialogue10]

# 3. Deploy a reasoning environment

In [ ]:
ontology_path: str = Path("/kaggle/input/rdfontology/data.rdf")

In [8]:
# TODO: load your ontology and create reasoner

from owlapy.owl_ontology import SyncOntology

ontology_path: str = "/kaggle/input/rdf-ontology/data.rdf"

load = SyncOntology(ontology_path)

reasoner = SyncReasoner(load)

print("Ontology loaded and reasoner ready")


Ontology loaded and reasoner ready


### Test querying code

- Testing if simple query works with our Reasoner setup
- Helper code for findng entity IRI based on rdfs: label (e.g. Rome)

In [ ]:
from rdflib import Graph

g = Graph()
g.parse("/kaggle/input/rdfontology/data.rdf")

def iris_by_label(label: str):
    q = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT DISTINCT ?s WHERE {{
      ?s rdfs:label "{label}" .
    }}
    """
    return [str(row.s) for row in g.query(q)]

print(iris_by_label("Rome"))

In [54]:
namespace = 'http://kai.cs.vu.nl/2024/situated-minor-project/hotel#'

expr_str = (
    'Hostel and hasFacility value Free_Wifi'
)
results = reasoner.instances(manchester_to_owl_expression(expr_str, namespace))

print(results)

{OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation436')), OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation165')), OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation708')), OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation720')), OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation794')), OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation196')), OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation397')), OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation100')), OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation716')), OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 

# 4. Instruct the LLM to produce the query or components of the query (e.g., keywords) against the KG

In [9]:
!sudo apt-get update && sudo apt-get install -y zstd

!curl -fsSL https://ollama.com/install.sh | sh

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]           
Get:5 https://cli.github.com/packages stable/main amd64 Packages [354 B]       
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,297 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,868 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InR

In [10]:
#Download ollama
# For Kaggle or Linux: download with this command, for Windows & Mac locally, download executable from website
!curl -fsSL https://ollama.com/install.sh | sh

import subprocess
process = subprocess.Popen("ollama serve", shell=True) #runs on a different thread

#Download Python library
!pip install ollama

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%                                    3.8%                                                  13.6%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIERSmd0u+ttXm6GbqR4cIkYrYUZ6xcmV3eWDkxl0ASX1



time=2026-01-15T07:43:07.975Z level=INFO source=routes.go:1614 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GGML_VK_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_REMOTES:[ollama.com] OLLAMA_SCHED_SP

In [11]:
# Import ollama & pull LLM
import ollama
!ollama pull llama3.2
model: str = "llama3.2"

[GIN] 2026/01/15 - 07:43:19 | 200 |      80.544µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ 

time=2026-01-15T07:43:19.786Z level=INFO source=download.go:177 msg="downloading dde5aa3fc5ff in 16 126 MB part(s)"


pulling manifest 
pulling dde5aa3fc5ff:   2% ▕                  ▏  44 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   4% ▕                  ▏  86 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   9% ▕█                 ▏ 186 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  14% ▕██                ▏ 280 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  16% ▕██                ▏ 329 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  22% ▕███               ▏ 436 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  26% ▕████              ▏ 533 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  29% ▕█████             ▏ 581 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  34% ▕██████            ▏ 684 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:  39% ▕███████           ▏ 791 MB/2.0 GB                  pulling manifest 
pulling dde5

time=2026-01-15T07:43:22.929Z level=INFO source=download.go:177 msg="downloading 966de95ca8a6 in 1 1.4 KB part(s)"


pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca

...

pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB              

time=2026-01-15T07:43:26.300Z level=INFO source=download.go:177 msg="downloading 56bb8bd477a5 in 1 96 B part(s)"


pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB              

time=2026-01-15T07:43:27.432Z level=INFO source=download.go:177 msg="downloading 34bb5ab01051 in 1 561 B part(s)"


pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         pullin

In [55]:
#Step 1: Write the instruction for the LLM - remember the overarching topic (assistance with hotels), as well as the fact that
# this step is meant to merely extract queries from the user input.

# Instruct LLM
instruction: str = """You are an AI assistant which is going to be used by users to query a Hospitality knowledge databased 
The goal for you is to translate the user's natural language statement/question into a description logic query using Manchester Syntax

Rules -
Output only Manchester Syntax query string without any unnecessary symbols (like \n etc). Don't include explanations or controversial text.
Exact words matter along with the casing such as "Hotel" and not "hotel"
For values such as city names use the 'value' keyword  with the city name inside double quotes.
Combine multiple conditions using 'and' 'or' and 'not'

Use only the classes and properties from the dataset in your query. this is our ontology schema:

Object properties:

diet

hasFacility

inCity

inCountry

location

nationality

nextTo

restaurantType

userRating

topObjectProperty

Classes

Airport

Camping_Site

City

Country

Diet

Facility

Hostel

Hotel

Landmark

Museum

Nationality

Neighbourhood

Restaurant

RestaurantType

TouristAttraction

Trainstation

UserRating

AccommodationEntities

LuxuryHotel

TransportationEntities

WaterBody

Individuals

1_stars

2_stars

3_stars

4_stars

5_stars

24h_front_desk

Swimming_Pool

Sauna

Airport_Shuttle

Private_Bathroom

Parking_Space

RestaurantInHotel

Free_Wifi

Examples - 
User: "I am looking for a hotel in Amsterdam"
Query: Hotel and inCity value "Amsterdam"

User: "Show me camping sites in Paris"
Query: CampingSite and inCity value "Paris"

User: "Show me camping site with airpot shuttle"
Query: CampingSite and hasFacility value Airport_Shuttle

User: "Find hotels with a rating of 5 stars"
Query: Hotel and has rating value "5-stars"
"""

In [56]:
#Step 2: Write a function that takes the model, instruction and one user question as input, runs the LLM and outputs its response
def question_to_query(instruction: str, question: str, model="llama3.2") -> str:
    '''
    This function is meant to use the instruction defined above to run the LLM in order to convert one user input
    question into a query for the ontology reasoner.
    Parameters: instruction (string), question (string), model version (string)
    Returns: LLM response (string)
    '''

    messages = [
        {'role': 'system', 'content': instruction},
        {'role': 'user', 'content': question}
    ]

    response = ollama.chat(
        model=model,
        messages=messages
    )

    output = response['message']['content']

    return output.strip()

In [57]:
#Step 3: Run the LLM for each example defined above

# Helper function
def find_between(s: str, start: str, end: str) -> str:
    return s.split(start)[1].split(end)[0]

queries = []

for item in dialogues:
   
    user_question = item['dialogue'] # This change has been made since we are using dictionaries and not strings in our dialogues
    print("User question:", user_question)
    print()
    result: str = question_to_query(instruction, user_question, model)
    # Possibly only extract the relevant parts
    print("Extracted query:", result)
    queries.append(result)
    print()

User question: List all hostels that offer free wifi

[GIN] 2026/01/15 - 08:29:29 | 200 |  400.828978ms |       127.0.0.1 | POST     "/api/chat"
Extracted query: Hostel and hasFacility value Free_Wifi

User question: List all hotels near a water body

[GIN] 2026/01/15 - 08:29:30 | 200 |  367.420025ms |       127.0.0.1 | POST     "/api/chat"
Extracted query: Manchester Syntax Query:
Hotel and topObjectProperty value WaterBody

User question: Find a restaurant that has traditional food

[GIN] 2026/01/15 - 08:29:30 | 200 |  324.127606ms |       127.0.0.1 | POST     "/api/chat"
Extracted query: RestaurantType and inDomain value "traditional_food"

User question: I want to live in a luxury hotel that has parking along with a swimming pool and sauna

[GIN] 2026/01/15 - 08:29:31 | 200 |  544.115085ms |       127.0.0.1 | POST     "/api/chat"
Extracted query: LuxuryHotel and hasFacility value Parking_Space and hasFacility value Swimming_Pool and hasFacility value Sauna

User question: Show me 4

In [58]:
queries[0]

'Hostel and hasFacility value Free_Wifi'

# 5. Use an LLM to summarize some result into a natural language response to the user.

In [65]:
def reason(query: str) -> list[str]:
    """
    Convert a query into an OWL expression and return inferred instance IRIs.
    If the reasoner fails, return ["No data found"].
    """
    namespace = 'http://kai.cs.vu.nl/2024/situated-minor-project/hotel#'

    try:
        results = reasoner.instances(
            manchester_to_owl_expression(query, namespace)
        )
    except Exception:
        return ["No data found"]

    iris: list[str] = []

    for ind in results:
        if hasattr(ind, "iri"):
            iris.append(str(ind.iri))
        else:
            iris.append(str(ind))

    return iris if iris else ["No data found"]


In [66]:
#Step 2: Instruct & run the LLM for the new task: transform the extracted knowledge into a natural language response based
# on the original question
def knowledge_to_response(question: str, knowledge: str, model="llama3.2"):
    '''
    This function is meant to write an instruction based on an item of extracted knowledge and the original user
    question, and run the LLM to summarize a response.
    '''
    print("="*70)
    print("Question:", question,"\n")
    messages = [
        {'role': 'system', 'content': "This is related knowledge"+knowledge},
        {'role': 'user', 'content': question}
    ]

    response = ollama.chat(
        model=model,
        messages=messages
    )

    output = response['message']['content']

    return output.strip()

In [59]:
print(queries[0])
reason(queries[0])

Hostel and hasFacility value Free_Wifi


{OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation100')),
 OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation103')),
 OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation104')),
 OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation107')),
 OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation109')),
 OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation11')),
 OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation112')),
 OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation113')),
 OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/hotel#', 'accomodation121')),
 OWLNamedIndividual(IRI('http://kai.cs.vu.nl/2024/situated-minor-project/h

In [ ]:
#Step 3: Combine everything: generate queries from the dialogues, extract knowledge from queries with the reasoner and
for i in range(len(dialogues)):
    print("reasoning")
    reasoner_result_str = "\n".join(reason(queries[i])) or "No results."
    print("finished reasoning")
    knowledge_result = knowledge_to_response(dialogues[i]['dialogue'], reasoner_result_str)
    print(knowledge_result)
    print("="*70)

reasoning
finished reasoning
Question: List all hostels that offer free wifi 

[GIN] 2026/01/15 - 08:46:27 | 200 |  9.414220808s |       127.0.0.1 | POST     "/api/chat"
I'd be happy to provide you with a list of popular hostels that offer free WiFi, but please note that this list is not exhaustive and is subject to change. Additionally, it's always best to check with the hostel directly for their current policies and availability.

Here are some well-known hostels that offer free WiFi:

**Europe:**

1. **Meininger Hostels**: With locations in Berlin, London, Paris, Amsterdam, and more, Meininger offers free WiFi throughout their properties.
2. **Generator Hostels**: Generator has a strong presence across Europe, with locations in London, Berlin, Lisbon, and more, all offering free WiFi.
3. **Safestay Hostels**: Safestay has hostels in London, Brighton, Bristol, and Amsterdam, among others, with free WiFi available throughout their properties.
4. **St Christopher's Inns**: St Christoph

# 6. Evaluate your LLM

In [ ]:
# TODO: your code to implement and demonstrate evaluation metrics
# Suggestions: comparison of generated queries with the queries manually created in examples.txt, Intersection Over Union,
# but you can be creative here